In [ ]:
import sys
sys.path.append('../../PlateTectonicTools/')

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pygplates
import pygmt

from gprm import ReconstructionModel
from gprm.datasets import Rocks, Reconstructions, Paleogeography, Geology
from gprm.utils.raster import to_anchor_plate
from gprm.utils.geometry import nearest_feature

import sys
#sys.path.append('/Users/simon/OneDrive/Andes_works//python/')
sys.path.append('../../andes_paleoelevation//python/')
import joyful_geochemistry as joy
import joyful_mapping as joymap
from time_series import lowess

import collections

%matplotlib inline
%load_ext autoreload
%autoreload 2


In [ ]:
df = pd.read_excel('../../../Downloads/DZ TE Final.xlsx', sheet_name='Filtered Database v1')

df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Longitude'], 
                                                      df['Latitude']), crs='EPSG:4326')

df


In [ ]:
plt.plot(df['Best Age'], df['Eu/Eu*'], '.')

df.Continent.value_counts()

In [ ]:
continents = ['North America', 'Europe', 'Asia',
              'South America', 'Antarctica', 'Oceana']

import matplotlib.ticker as mticker

def my_boxplot(df_binned, ax, colname,
               color='black', widths=50):
    
    midpoints = [key for key in list(df_binned.groups.keys())]

    median_elevations = []
    elevation_violins = []
    for group in df_binned:
        if not group[1].empty:
            median_elevations.append(group[1][colname].median())
            elevation_violins.append(group[1][colname])
            group[0]
        else:
            median_elevations.append(np.nan)
            elevation_violins.append([np.nan,np.nan])

    parts = ax.boxplot(elevation_violins, positions=midpoints, widths=widths, 
                           patch_artist=True, manage_ticks=False, showfliers=False)
    for pc in parts['boxes']:
        pc.set_facecolor(color)
        pc.set_alpha(0.5)
        pc.set_edgecolor(color)

        
time_bin_size = 25
ncols = 3

fig = plt.figure(figsize=(16,8))

gs = fig.add_gridspec(nrows=6, ncols=ncols, wspace=0.12, hspace=0.0, height_ratios=[2,2,5,0.8,2,5])

for i,continent in enumerate(continents):
    
    print(continent)
    
    if i>ncols-1:
        row=5
    else:
        row=2
    ax = fig.add_subplot(gs[row, np.mod(i,ncols)])

    tmp = df[df.Continent.str.contains(continent)].reset_index()
    tmp = tmp.dropna(subset=['Eu/Eu*']).reset_index()
    #tmp['bin_age'] = np.round(tmp['Age']/time_bin_size)*time_bin_size
    tmp['bin_age'] = joy.assign_bin_ages(tmp['Best Age'], time_bin_size)

    tmp['thickness'] = 84.2 * tmp['Eu/Eu*'] + 24.5

    my_boxplot(tmp.groupby(by='bin_age'), ax, 'thickness', widths=time_bin_size*0.8)
    
    ax.set_xlim(0,1000)
    ax.set_ylim(25,90)
    ax.grid(axis='y')
    if row==5:
        ax.set_xlabel('Age [Ma]')
    else:
        ax.set_xticklabels([])
    if np.mod(i,ncols)==0:
        ax.set_ylabel('Thickness [km]')
    elif np.mod(i,ncols)==ncols-1:
        ax.yaxis.tick_right()
        ax.yaxis.set_label_position("right")
        ax.set_ylabel('Elevation [km]')
        ticks_loc = ax.get_yticks().tolist()
        ax.yaxis.set_major_locator(mticker.FixedLocator(ticks_loc))
        ax.set_yticklabels(['{:0.1f}'.format(t/1000.) for t in joy.thickness2elevation(1000*ax.get_yticks())])
    else:
        ax.set_yticklabels([])
    
    ax = fig.add_subplot(gs[row-1, np.mod(i,ncols)])
    ax.hist(tmp.bin_age, 
            bins=np.arange(0,1001,time_bin_size),
            color='gray')
    ax.set_xticks([])
    ax.set_xlim(0,1000)
    ax.set_ylim(1,20000)
    ax.set_yscale('log')
    ax.grid(which='both', axis='y', linestyle='--')
    ax.set_title('{:s} '.format(continent), loc='right', y=1.0, pad=-15)
    if np.mod(i,ncols)!=0:
        ax.set_yticklabels([])


plt.savefig('../images/boxplots/europium_puetz_thickness_time_series_by_continent_{:0.0f}Myr_bin.png'.format(time_bin_size), 
            dpi=600,
            bbox_inches='tight')
plt.show()


#sns.boxplot(x='bin_age', y='thickness', data=tmp)

#plt.plot(tmp['Age'], 84.2*tmp['Eu/Eu*zr'] + 24.5, '.')
#plt.xlim(0,1000)
#plt.ylim(20,90)


In [ ]:
time_bin_size = 25


gs = fig.add_gridspec(nrows=2, ncols=1, wspace=0.12, hspace=0.0, height_ratios=[1.5,5])

fig = plt.figure(figsize=(7,6))

ax = fig.add_subplot(gs[1,0])
tmp = df.copy(deep=True)
tmp = tmp.dropna(subset=['Eu/Eu*']).reset_index() 
#tmp['bin_age'] = np.round(tmp['Age']/time_bin_size)*time_bin_size
#tmp['bin_age'] = (np.round((tmp['Age']-time_bin_size/2)/time_bin_size)*time_bin_size) + (time_bin_size/2)
tmp['bin_age'] = joy.assign_bin_ages(tmp['Best Age'], time_bin_size)

tmp['thickness'] = 84.2 * tmp['Eu/Eu*'] + 24.5

my_boxplot(tmp.groupby(by='bin_age'), ax, 'thickness', widths=time_bin_size*0.8)

ax.set_xlim(0,1000)
ax.set_ylim(20,90)
ax.grid(axis='y')

ax.set_xlabel('Age [Ma]')
ax.set_ylabel('Thickness [km]')
axr = ax.twinx()
axr.yaxis.tick_right()
axr.set_ylim(ax.get_ylim())
axr.yaxis.set_label_position("right")
axr.set_ylabel('Elevation [km]')
ticks_loc = axr.get_yticks().tolist()
axr.yaxis.set_major_locator(mticker.FixedLocator(ticks_loc))
print(axr.get_yticks())
axr.set_yticklabels(['{:0.1f}'.format(t/1000.) for t in joy.thickness2elevation(1000*axr.get_yticks())])


ax = fig.add_subplot(gs[0,0])
ax.hist(tmp.bin_age, 
        bins=np.arange(0,1001,time_bin_size),
        color='gray')
ax.set_xticks([])
ax.set_xlim(0,1000)
#ax.set_ylim(0,20000)
ax.set_yscale('log')
ax.grid(which='both', axis='y', linestyle='--')

plt.savefig('../images/boxplots/europium_puetz_thickness_time_series_{:0.0f}Myr_bin.png'.format(time_bin_size), dpi=600)
plt.show()


In [ ]:
orogen_names = [
    #('Cordilleran', 'Cordilleran-SAM', 'South America'),
    ('Cordilleran', 'Cordilleran-NAM', 'North America'),
    #('Cordilleran', 'Cordilleran-Asia', 'Asia'),
    ('Cordilleran', 'Cordilleran-Ant', ['Australia', 'Zealandia', 'Antarctica']),
    ('Terra Australis', 'Terra Australis', None),
    ('Solonker', 'Solonker', None),
    ('Indosinian', 'Indosinian', None),
    ('Variscan-Hercynian', 'Variscan-Hercynian', None),
    #'Uralian',
    ('Mongol', 'Mongol', None),
    #('Kazakh', 'Kazakh', None),
    ('Caledonia', 'Caledonia', None),
    ('Alpine-Himalayan', 'Alpine-Himalayan-Europe', 'Europe'),
    ('Alpine-Himalayan', 'Alpine-Himalayan-Asia', ['Asia', 'India and Middle East']),
]


for orogen_name in orogen_names:

    df_sel = joymap.select_orogens(
            df, 
            gdf=None, 
            orogen_names=[orogen_name[0]],
            continent_names=orogen_name[2]
        )

    
    gs = fig.add_gridspec(nrows=2, ncols=1, wspace=0.12, hspace=0.0, height_ratios=[1.5,5])

    fig = plt.figure(figsize=(7,6))

    ax = fig.add_subplot(gs[1,0])
    tmp = df_sel.copy(deep=True)
    tmp = tmp.dropna(subset=['Eu/Eu*']).reset_index() 
    #tmp['bin_age'] = np.round(tmp['Age']/time_bin_size)*time_bin_size
    #tmp['bin_age'] = (np.round((tmp['Age']-time_bin_size/2)/time_bin_size)*time_bin_size) + (time_bin_size/2)
    tmp['bin_age'] = joy.assign_bin_ages(tmp['Best Age'], time_bin_size)

    tmp['thickness'] = 84.2 * tmp['Eu/Eu*'] + 24.5

    my_boxplot(tmp.groupby(by='bin_age'), ax, 'thickness', widths=time_bin_size*0.8)

    ax.set_xlim(0,1000)
    ax.set_ylim(20,90)
    ax.grid(axis='y')

    ax.set_xlabel('Age [Ma]')
    ax.set_ylabel('Thickness [km]')
    axr = ax.twinx()
    axr.yaxis.tick_right()
    axr.set_ylim(ax.get_ylim())
    axr.yaxis.set_label_position("right")
    axr.set_ylabel('Elevation [km]')
    ticks_loc = axr.get_yticks().tolist()
    axr.yaxis.set_major_locator(mticker.FixedLocator(ticks_loc))
    print(axr.get_yticks())
    axr.set_yticklabels(['{:0.1f}'.format(t/1000.) for t in joy.thickness2elevation(1000*axr.get_yticks())])


    ax = fig.add_subplot(gs[0,0])
    ax.hist(tmp.bin_age, 
            bins=np.arange(0,1001,time_bin_size),
            color='gray')
    ax.set_xticks([])
    ax.set_xlim(0,1000)
    #ax.set_ylim(0,20000)
    ax.set_yscale('log')
    ax.grid(which='both', axis='y', linestyle='--')

    plt.show()
    
    
#df_sel


In [ ]:
df['Best Age'].hist(bins=np.arange(0,600,10))